In [9]:
from collections import deque
import pickle
import random

def bfs(tree, start):
    visited = set([start])
    queue = deque([(start, 0)])
    distances = {}
    
    while queue:
        node, distance = queue.popleft()
        distances[node] = distance
        
        for neighbor in tree[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, distance + 1))
    
    return distances

def calculate_m_values(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        for h in range(len(tree)):
            if len([x for x in distances.values() if x <= h]) >= k:
                m_values.append(h)
                break
    
    return m_values

def greedy_labeling(tree, k, m_values):
    nodes = sorted(range(len(tree)), key=lambda x: m_values[x])
    labels = [None] * len(tree)
    
    step = 0
    while True:
        for node in nodes:
            if labels[node] is None:
                available_labels = set(range(k))
                distances = bfs(tree, node)
                neighbors = [x for x in distances if distances[x] <= m_values[node]]
                neighbors_p = neighbors[1:]
                random.shuffle(neighbors_p)
                neighbors[1:] = neighbors_p
                
                for neighbor in neighbors:
                    if labels[neighbor] in available_labels:
                        available_labels.remove(labels[neighbor])
                    if not available_labels:
                        break
                
                available_labels = list(available_labels)
                random.shuffle(available_labels)
                
                if available_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] is None:
                            labels[neighbor] = available_labels.pop()
                        if not available_labels:
                            break
                        
                unique_labels = set()
                if available_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] in unique_labels:
                            labels[neighbor] = available_labels.pop()
                            if not available_labels:
                                break
                        unique_labels.add(labels[neighbor])
            else:
                missing_labels = set(range(k))
                distances = bfs(tree, node)
                neighbors = [x for x in distances if distances[x] <= m_values[node]]
                neighbors_p = neighbors[1:]
                random.shuffle(neighbors_p)
                neighbors[1:] = neighbors_p
                
                for neighbor in neighbors:
                    if labels[neighbor] in missing_labels:
                        missing_labels.remove(labels[neighbor])
                    if not missing_labels:
                        break
                
                missing_labels = list(missing_labels)
                random.shuffle(missing_labels)
                
                if not missing_labels:
                    continue
                else:
                    for neighbor in neighbors:
                        if labels[neighbor] is None:
                            labels[neighbor] = missing_labels.pop()
                        if not missing_labels:
                            break
                        
                    unique_labels = set()
                    if missing_labels:
                        for neighbor in neighbors:
                            if labels[neighbor] in unique_labels:
                                labels[neighbor] = missing_labels.pop()
                                if not missing_labels:
                                    break
                            unique_labels.add(labels[neighbor])
            
        count = 0
        for node in nodes:
            missing_labels = set(range(k))
            distances = bfs(tree, node)
            neighbors = [x for x in distances if distances[x] <= m_values[node]+step]

            for neighbor in neighbors:
                if labels[neighbor] in missing_labels:
                    missing_labels.remove(labels[neighbor])
                if not missing_labels:
                    break

            if not missing_labels:
                count += 1
            
        if count == len(tree):
            break
        else:
            step += 0.01

    for i in range(len(labels)):
        if labels[i] is None:
            labels[i] = 0

    return labels

def load_input(file_adjlist, file_k_values):
    with open(file_adjlist, 'rb') as f:
        adj_list_of_trees = pickle.load(f)
    with open(file_k_values, 'rb') as f:
        k_values = pickle.load(f)
    
    return zip(adj_list_of_trees, k_values)    

def save_output(file_name, output_list):
    with open(file_name, 'wb') as f:
        pickle.dump(output_list, f)

def main():
    # Load input instances
    # data_input = load_input('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values')
    # data_input = load_input('Small_Examples_of_AdjLists_of_Trees', 'Small_Examples_of_k_values')
    data_input = load_input('Medium_Examples_of_AdjLists_of_Trees', 'Medium_Examples_of_k_values')
    # data_input = load_input('Large_Examples_of_AdjLists_of_Trees', 'Large_Examples_of_k_values')

    # Run algorithm and save results
    
    results = []
    for tree, k in data_input:
        m_values = calculate_m_values(tree, k)
        labels = greedy_labeling(tree, k, m_values)
        results.append(labels)

    # save_output('example_solutions', results)
    # save_output('small_solutions', results)
    save_output('medium_solutions', results)
    # save_output('large_solutions', results)

if __name__ == '__main__':
    main()

In [10]:
def bfs(tree, start):
    visited = set()
    queue = [(start, 0)]
    distances = {}

    while queue:
        current, depth = queue.pop(0)
        visited.add(current)
        distances[current] = depth
        for neighbor in tree[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth + 1))

    return distances

def r(tree, k, solution):
    r_values = []
    for node, label in enumerate(solution):
        label_set = set()
        distances = bfs(tree, node)
        h = 0
        while len(label_set) < k and h < len(tree):
            label_set.update({solution[i] for i, dist in distances.items() if dist <= h})
            h += 1
        r_values.append(h - 1)
    return r_values

def m(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        h = 0
        while len([1 for i, dist in distances.items() if dist <= h]) < k:
            h += 1
        m_values.append(h)
    return m_values

def compute_proximity_ratios(file_adjlist, file_k_values, file_solutions):
    input_instances = load_input(file_adjlist, file_k_values)
    solutions = load_solutions(file_solutions)

    proximity_ratios = []
    for (tree, k), solution in zip(input_instances, solutions):
        r_values = r(tree, k, solution)
        m_values = m(tree, k)

        ratios = [rv / mv for rv, mv in zip(r_values, m_values)]
        max_ratio = max(ratios)
        proximity_ratios.append(max_ratio)

    return proximity_ratios

def load_solutions(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def main():
    # Compute proximity ratios for small solutions
    # proximity_ratios = compute_proximity_ratios('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values', 'example_solutions')
    # proximity_ratios = compute_proximity_ratios('Small_Examples_of_AdjLists_of_Trees', 'Small_Examples_of_k_values', 'small_solutions')
    proximity_ratios = compute_proximity_ratios('Medium_Examples_of_AdjLists_of_Trees', 'Medium_Examples_of_k_values', 'medium_solutions')
    # proximity_ratios = compute_proximity_ratios('Large_Examples_of_AdjLists_of_Trees', 'Large_Examples_of_k_values', 'large_solutions')
    print("Proximity ratios:", proximity_ratios)
    print("Maximum ratio:", max(proximity_ratios))

if __name__ == '__main__':
    main()
    # sols = load_solutions('small_solutions')
    # for sol in sols:
    #     print(sol)

Proximity ratios: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Maximum ratio: 1.0


In [11]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_graph(adj_list):
    G = nx.Graph()

    # Adding nodes and edges to the graph
    for node, neighbors in enumerate(adj_list):
        G.add_node(node)
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

    # Drawing the graph without labels
    nx.draw(G, with_labels=True, node_color="lightblue")
    plt.show()

In [12]:

tree = list(load_input('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values'))
draw_graph(tree[4][0])
print(tree[4][1])
print(load_solutions('example_solutions')[4])
print(list(range(len(tree[4][0]))))
print(m(tree[4][0], tree[4][1]))
print(r(tree[4][0], tree[4][1], load_solutions('example_solutions')[4]))

TypeError: '_AxesStack' object is not callable

<Figure size 640x480 with 0 Axes>

In [ ]:
distances = bfs(tree[4][0], 9)
neighbors = [x for x in distances if distances[x] <= 2]
neighbors

[9, 6, 11, 5]

In [ ]:
detail_labeling(tree[4][0], tree[4][1], m(tree[4][0], tree[4][1]))

[1, 5, 0, 2, 3, 4, 6, 7, 8, 9, 10, 11]
1 is not labeled
neighbors:  [1, 0, 2, 3, 4]
Avail:  {0, 1, 2, 3}
1 :  0
0 :  1
2 :  2
3 :  3
Avail:  set()
5 is not labeled
neighbors:  [5, 3, 6, 7, 8]
Avail:  {0, 1, 2}
5 :  0
6 :  1
7 :  2
Avail:  set()
0 is labeled
neighbors:  [0, 1, 2, 3, 4]
missing:  set()
2 is labeled
neighbors:  [2, 1, 0, 3, 4]
missing:  set()
3 is labeled
neighbors:  [3, 1, 5, 0, 2, 4, 6, 7, 8]
missing:  set()
4 is not labeled
neighbors:  [4, 1, 0, 2, 3]
Avail:  set()
Avail:  set()
6 is labeled
neighbors:  [6, 5, 9, 3, 7, 8, 11]
missing:  set()
7 is labeled
neighbors:  [7, 5, 10, 3, 6, 8]
missing:  set()
8 is not labeled
neighbors:  [8, 5, 3, 6, 7]
Avail:  set()
Avail:  set()
9 is not labeled
neighbors:  [9, 6, 11, 5]
Avail:  {2, 3}
9 :  2
11 :  3
Avail:  set()
10 is not labeled
neighbors:  [10, 7, 5, 3, 6, 8]
Avail:  set()
Avail:  set()
11 is labeled
neighbors:  [11, 9, 6, 5]
missing:  set()


In [ ]:
def detail_labeling(tree, k, m_values):
    nodes = sorted(range(len(tree)), key=lambda x: m_values[x])
    labels = [None] * len(tree)
    
    # flag = True
    # while flag:
    print(nodes)
    for node in nodes:
        if labels[node] is None:
            print(node, "is not labeled")
            available_labels = set(range(k))
            distances = bfs(tree, node)
            neighbors = [x for x in distances if distances[x] <= m_values[node]]
            print("neighbors: ", neighbors)
            
            for neighbor in neighbors:
                if labels[neighbor] in available_labels:
                    available_labels.remove(labels[neighbor])
                if not available_labels:
                    break
            
            print("Avail: ", available_labels)
            if available_labels:
                for neighbor in neighbors:
                    if labels[neighbor] is None:
                        labels[neighbor] = available_labels.pop()
                        print(neighbor, ": ", labels[neighbor])
                    if not available_labels:
                        break
            print("Avail: ", available_labels)
            unique_labels = set()
            if available_labels:
                for neighbor in neighbors:
                    if labels[neighbor] in unique_labels:
                        labels[neighbor] = available_labels.pop()
                        if not available_labels:
                            break
                    unique_labels.add(labels[neighbor])
        else:
            print(node, "is labeled")
            missing_labels = set(range(k))
            distances = bfs(tree, node)
            neighbors = [x for x in distances if distances[x] <= m_values[node]]
            print("neighbors: ", neighbors)
            for neighbor in neighbors:
                if labels[neighbor] in missing_labels:
                    missing_labels.remove(labels[neighbor])
                if not missing_labels:
                    break
            print("missing: ", missing_labels)
            if not missing_labels:
                continue
            else:
                for neighbor in neighbors:
                    if labels[neighbor] is None:
                        labels[neighbor] = missing_labels.pop()
                    if not missing_labels:
                        break
                    
                unique_labels = set()
                if missing_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] in unique_labels:
                            labels[neighbor] = missing_labels.pop()
                            if not missing_labels:
                                break
                        unique_labels.add(labels[neighbor])
    return labels

In [ ]:
ne = list(range(10))
ne_p = ne[1:]
random.shuffle(ne_p)
ne[1:] = ne_p
print(ne)

[0, 5, 2, 3, 4, 9, 6, 7, 1, 8]
